## 1- Library and Data Imports

In [55]:
import numpy as np
import pandas as pd
import time

# for text cleaning and preprocessing
import re
from nltk.corpus import stopwords
import string 
from sklearn.feature_extraction.text import TfidfVectorizer

import sys
sys.path.append('../Src')
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
from preprocessing.data_clean import clean_text

In [35]:
SE_df = pd.read_csv('../Data/processed/SE_data1.csv', keep_default_na=False, encoding='utf-8-sig')

In [36]:
SE_df

,_id,url,title,text,published_at,article_type,tags,text_clean,summary_clean,title_clean
0,633fdfd13ffae8229d05cb33,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,التربية: تحويل 42 مدرسة إلى نظام الفترتين واست...,أكدت أمين عام وزارة التربية والتعليم للشؤون ...,2022-10-06 07:33:00,News,"['التربية والتعليم', 'وزارة التربية والتعليم']",اكد امين عام وزاره تربيه تعليم شان مالي اداري ...,,تربيه تحويل 42 مدرسه الي نظام فتره استءجار 1...
1,633fdfd13ffae8229d05cb34,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,تكريما للمعلمين زيادة منح أبناء المعلمين 550 م...,احتفلت وزارة التربية والتعليم بيوم المعلم بت...,2022-10-05 11:46:00,News,"['مكرمة أبناء المعلمين', 'وزارة التربية والتعل...",احتفل وزاره تربيه تعليم يوم معلم تكريم نخبه مع...,,تكريم معلم زياده منح ابناء معلم 550 مقعد اضافي...
2,633fdfd13ffae8229d05cb35,https://husna.fm/%D9%85%D9%84%D9%81%D8%A7%D8%A...,الغاز الروسي يضع أوروبا خلال الشتاء المقبل أما...,يشهد العالم أول أزمة طاقة عالمية حقيقية في الت...,2022-08-22 07:17:00,News,"['الغاز الروسي', 'أوروبا', 'أزمة الطاقة']",شهد عالم اول ازمه طاقه عالم حقيقي تاريخ . تنام...,,غاز روسي وضع اوربا خلال شتاء مقبل امام اختبار ...
3,633fdfd13ffae8229d05cb36,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,تفاصيل دوام المدارس بعد قرار تثبيت التوقيت الصيفي,كشفت أمين عام وزارة التربية والتعليم للشؤون ال...,2022-10-06 05:29:00,News,"['التربية والتعليم', 'التوقيت الصيفي', 'المدارس']",كشف امين عام وزاره تربيه تعليم شان مالي اداري ...,,تفصيل دوام مدرسه قرار تثبيت توقيت صيفي
4,633fdfd13ffae8229d05cb37,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...,العمل: عطلة المولد النبوي الشريف تشمل القطاع ا...,أكدت وزارة العمل أن العطل الرسمية تكون مأجور...,2022-10-06 07:46:00,News,"['وزارة العمل', 'المولد النبوي']",اكد وزاره عمل ان عطل رسمي ماجور عامل مءسسه قطا...,,عمل عطله مولد نبوي شريف شمل قطاع خاص
...,...,...,...,...,...,...,...,...,...,...
5079,633ff10f322e7883822052b3,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,الديمقراطية والتقاء الخصوم,نتحدث هنا عن القوى المؤثرة على الأرض العربية ا...,2022-09-20 21:00:00,Blog,['الديموقراطية'],تحدث قوه مءثر علي ارض عربي اصبح سبخ مالح بيءه ...,,ديموقراطي التقاء خصم
5080,633ff10f322e7883822052b4,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,علموا أولادكم ألوان الطيف السبعة ومعانيها,سبحان من خلق الكون وأبدع في تصوريه، فجعل السما...,2022-09-21 12:18:00,Blog,"['ألوان الطيف السبعة', 'الشذوذ الجنسي']",خلق كون بدع صور ، سماء درجه ، ارض منزل ، ايه ا...,,ولد لون طيف معني
5081,633ff10f322e7883822052b7,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,تدين الخلاص الفردي وفرص النجاح,حالة من الصراع الصفري بين الأنظمة والإسلام الس...,2022-10-02 21:00:00,Blog,"['الإسلام', 'الإسلام السياسي']",حال صراع صفر ي نظام اسلام سياسي تولد ربيع عربي...,,ادان خلاص فردي فرصه نجاح
5082,633ff110322e7883822052b9,https://husna.fm/%D9%85%D9%82%D8%A7%D9%84%D8%A...,السر وراء الانخفاض الملحوظ في أسعار الأجهزة ال...,جدل يسود الشارع الأردني مجددا إثر التخفيضات ال...,2022-10-04 21:00:00,Blog,['تخفيضات'],جدل ساد شارع اردني مجدد اثر تخفيض كبير علي سعر...,,سر وراء انخفاض ملحوظ سعر جهاز كهرباءي


In [3]:
docs_to_store = SE_df[['url', 'title', 'text', 'tags']]
docs_to_store = docs_to_store.to_dict('records')

## 2- Connection 

In [56]:
import logging
def connect_elasticsearch():
    _es = None
    ELASTIC_PASSWORD = "oC+QPrfW=4B8GpDC_1hr"
    _es = Elasticsearch("https://localhost:9200", 
                   ca_certs='../Certs/http_ca.crt',
                   basic_auth=("elastic", ELASTIC_PASSWORD))
    if _es.ping():
        print('Yay Connect')
    else:
        print('Awww it could not connect!')
    return _es

In [57]:
es = connect_elasticsearch()

Yay Connect


In [58]:
# from datetime import datetime
# from elasticsearch import Elasticsearch


# ELASTIC_PASSWORD = "oC+QPrfW=4B8GpDC_1hr"
# es = Elasticsearch("https://localhost:9200", 
# #                    client_cert='a42181244c443b87e84571fb7d9b630ef550537de80636eac298e7ec8abbde33',
# #                    verify_certs=True,
#                    ca_certs='../Certs/http_ca.crt',
#                    basic_auth=("elastic", ELASTIC_PASSWORD))

# es.ping()

In [59]:
def create_index(es_object, index_name='SE_SHAI'):
    created = False
    # index settings
    settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "members": {
                "dynamic": "strict",
                "properties": {
                    "url": {
                        "type": "text"
                    },
                    "title": {
                        "type": "text"
                    },
                    "text": {
                        "type": "text"
                    },
                    "tags": {
                        "type": "text"
                    },
                }
            }
        }
    }
    
    try:
        if not es_object.indices.exists(index=index_name):
            # Ignore 400 means to ignore "Index Already Exist" error.
            es_object.indices.create(index=index_name, ignore=400, body=settings)
            print('Created Index')
        else:
            print('index exists')
        created = True
    except Exception as ex:
        print(str(ex))
    finally:
        return created

In [60]:
print(f"index created: {create_index(es, 'se_shai')}")

index exists
index created: True


In [61]:
def store_record(elastic_object, index_name, record, record_id):
    try:
        outcome = elastic_object.index(index=index_name, id=record_id, document=record)
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex))

In [62]:
for i, doc in enumerate(docs_to_store):
    store_record(es, 'se_shai', doc, i)

NameError: name 'docs_to_store' is not defined

In [63]:
def get_records(elastic_object, index_name, record_id):
    try:
        resp = elastic_object.get(index=index_name, id=record_id)
        return resp
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex))

In [16]:
resp_ret = get_records(es, 'se_shai', 1)
print(resp_ret)

{'_index': 'se_shai', '_id': '1', '_version': 1, '_seq_no': 1, '_primary_term': 1, '_ignored': ['text.keyword'], 'found': True, '_source': {'url': 'https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%B2%D9%8A%D8%A7%D8%AF%D8%A9-%D9%85%D9%86%D8%AD-%D8%A3%D8%A8%D9%86%D8%A7%D8%A1-%D8%A7%D9%84%D9%85%D8%B9%D9%84%D9%85%D9%8A%D9%86', 'title': 'تكريما للمعلمين زيادة منح أبناء المعلمين 550 مقعدا إضافيا بالجامعات', 'text': 'احتفلت  وزارة التربية والتعليم  بيوم المعلم بتكريمها نخبة من المعلمات والمعلمين في مختلف مديريات التربية والتعليم بالمملكة خلال الحفل الذي أقامته اليوم في مدرسة كنجز أكاديمي. أمين عام الوزارة للشؤون التعليمية الدكتور نواف العجارمة، أعلن خلال الاحتفال عن زيادة منح الطلبة أبناء المعلمين بنحو 550 مقعدا إضافيا في الجامعات الأردنية؛ تكريما من الوزارة للمعلمين. واستذكر العجارمة بهذه المناسبة الراحلين الأوائل من المعلمين الذين ما زالوا بيننا لإسهاماتهم في بناء ونهضة الأردن، مشيدا بدورهم في البناء ودورهم الكبير في بناء الأجيال المؤهلة والمدربة التي تنهض بعملية التطوير والتحديث في مسيرة الوط

In [64]:
# refereshing indices

# es.indices.refresh(index="se_shai")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [28]:
def search(es_object, index_name, search):
    res = es_object.search(index=index_name, query=search)

- for search, BM25 similarity is applied for scoring matched documents.
- scoring / ranking model: **BM25**, which is a TF/IDF based similarity that has built-in tf normalization.
    - other similarity ranking approaches: DFR similarity, DFI similarity, IB similarity, LM Dirichlet similarity, LM Jelinek Mercer similarity, Scripted similarity (Custom).  

In [29]:
q = 'مولد النبي'

resp = es.search(index="se_shai", query=
                 {"multi_match": 
                  { 
                      "query": q, 
                      "fields": [ "title", "text", "tags"] 
                  }
                 }
                )

# list of documents with scores
resp['hits']['hits']
# print("Got %d Hits:" % resp['hits']['total']['value'])
# print("Got %d Hits:" % resp['hits']['total']['value'])
# for hit in resp['hits']['hits']:
#     print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

[{'_index': 'se_shai',
  '_id': '2152',
  '_score': 8.810332,
  '_ignored': ['text.keyword', 'url.keyword'],
  '_source': {'url': 'https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%B0%D9%83%D8%B1%D9%89-%D8%A7%D9%84%D9%87%D8%AC%D8%B1%D8%A9-%D8%A7%D9%84%D9%86%D8%A8%D9%88%D9%8A%D8%A9-%D8%AF%D8%B1%D9%88%D8%B3-%D9%86%D8%B3%D8%AA%D9%84%D9%87%D9%85-%D9%85%D9%86%D9%87%D8%A7-%D8%A7%D9%84%D8%B9%D8%A8%D8%B1-%D9%81%D9%8A-%D8%A8%D9%86%D8%A7%D8%A1-%D8%A7%D9%84%D8%AF%D9%88%D9%84%D8%A9',
   'title': 'ذكرى الهجرة النبوية دروس نستلهم منها العبر في بناء الدولة',
   'text': 'تأتي ذكرى الهجرة النبوية الشريفة التي تصادف في الثَّلاثين من تمّوز 2022 ، الموافق للأوَّل من محرَّم لسنة 1444 هجريَّة؛ لتعطينا الدروس والعبر في الصبر والتكاتف في بناء الدولة وتثبيت أركان اقتصادها وقوانينها. وتحيي الهجرة النبوية العديد من المعاني السامية، حيث كانت مفتاحا لبناء الدولة، وغيرت مجرى التاريخ في النهج الإسلامي ونشر الإسلام، \xa0وأهم ما ميزها المؤاخاة بين المهاجرين الذين هاجروا من ديارهم استجابة للنداء الالهي الرباني، فكان الرسول

In [17]:
i_es = IndicesClient(es)

In [23]:
# get index
i_es.get(index='se_shai*')

ObjectApiResponse({})

In [ ]:
# delete index 
i_es.delete(index='se_shai')


In [153]:
es.close()

## Building Search Functionality

In [4]:
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient

In [5]:
def connect_elasticsearch():
    _es = None
    ELASTIC_PASSWORD = "oC+QPrfW=4B8GpDC_1hr"
    _es = Elasticsearch("https://localhost:9200", 
                   ca_certs='../Certs/http_ca.crt',
                   basic_auth=("elastic", ELASTIC_PASSWORD))
    if _es.ping():
        print('Yay Connect')
    else:
        print('Awww it could not connect!')
    return _es

In [6]:
es = connect_elasticsearch()

Yay Connect


In [54]:
q = 'مقال'
q_cleaned = clean_text(q)
q_cleaned

'مقال'

In [32]:
query = {"multi_match": 
                  { 
                      "query":  q+'*', 
                      "fields": [ "title", "text", "tags"] 
                  }
}

resp = es.search(index="se_shai", query=query)

urls = []
titles = []
texts = []
tags = []

# list of documents with scores
for hit in resp['hits']['hits']:
    urls.append(hit['_source']['url'])
    titles.append(hit['_source']['title'])
    texts.append(hit['_source']['text'])
    tags.append(hit['_source']['tags'])


{'_index': 'se_shai', '_id': '3140', '_score': 7.700178, '_ignored': ['text.keyword', 'url.keyword'], '_source': {'url': 'https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%A7%D9%84%D8%B3%D9%82%D8%A7%D8%B1-%D9%84%D8%AD%D8%B3%D9%86%D9%89-%D8%A7%D9%84%D8%AF%D9%8A%D9%88%D8%A7%D9%86-%D8%A7%D9%84%D9%85%D9%84%D9%83%D9%8A-%D8%A7%D8%AA%D8%B5%D9%84-%D8%A8%D9%8A-%D9%88%D8%AA%D9%83%D9%81%D9%84-%D8%A8%D9%83%D9%84%D9%81%D8%A9-%D8%B9%D9%84%D8%A7%D8%AC%D9%8A', 'title': 'السقار لحسنى: الديوان الملكي اتصل بي وتكفل بكلفة علاجي', 'text': 'قال الفنان متعب السقار لحسنى أنه تلقى اتصالاً هاتفياً من الديوان الملكي للاطمئنان على صحته، وإبلاغه بتغطية تكاليف عمليتي القسطرة\xa0 في قدميه التي أجراها في مستشفى الملك المؤسس، كما أوضح السقار أن الديوان أوعز بتسهيل إجراءات جراحة الكلى في حال توفر متبرع وفق القوانين المعمول بها في الأردن. على إثر ذلك، ذكر رئيس الوزراء بشر الخصاونة إن ولي العهد سمو الأمير الحسين بن عبد الله الثاني، أوعز بتقديم العناية الصحية اللازمة للفنان متعب السقار. جاء ذلك في جلسة مجلس النواب اليوم، حيث 